<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 2*
# Convolutional Neural Networks (CNNs)

# Assignment

- <a href="#p1">Part 1:</a> Pre-Trained Model
- <a href="#p2">Part 2:</a> Custom CNN Model
- <a href="#p3">Part 3:</a> CNN with Data Augmentation


You will apply three different CNN models to a binary image classification model using Keras. Classify images of Mountains (`./data/mountain/*`) and images of forests (`./data/forest/*`). Treat mountains as the postive class (1) and the forest images as the negative (zero). 

|Mountain (+)|Forest (-)|
|---|---|
|![](./data/mountain/art1131.jpg)|![](./data/forest/cdmc317.jpg)|

The problem is realively difficult given that the sample is tiny: there are about 350 observations per class. This sample size might be something that you can expect with prototyping an image classification problem/solution at work. Get accustomed to evaluating several differnet possible models.

# Pre - Trained Model
<a id="p1"></a>

Load a pretrained network from Keras, [ResNet50](https://tfhub.dev/google/imagenet/resnet_v1_50/classification/1) - a 50 layer deep network trained to recognize [1000 objects](https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt). Starting usage:

```python
import numpy as np

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D()
from tensorflow.keras.models import Model # This is the functional API

resnet = ResNet50(weights='imagenet', include_top=False)

```

The `include_top` parameter in `ResNet50` will remove the full connected layers from the ResNet model. The next step is to turn off the training of the ResNet layers. We want to use the learned parameters without updating them in future training passes. 

```python
for layer in resnet.layers:
    layer.trainable = False
```

Using the Keras functional API, we will need to additional additional full connected layers to our model. We we removed the top layers, we removed all preivous fully connected layers. In other words, we kept only the feature processing portions of our network. You can expert with additional layers beyond what's listed here. The `GlobalAveragePooling2D` layer functions as a really fancy flatten function by taking the average of each of the last convolutional layer outputs (which is two dimensional still). 

```python
x = res.output
x = GlobalAveragePooling2D()(x) # This layer is a really fancy flatten
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(res.input, predictions)
```

Your assignment is to apply the transfer learning above to classify images of Mountains (`./data/mountain/*`) and images of forests (`./data/forest/*`). Treat mountains as the postive class (1) and the forest images as the negative (zero). 

Steps to complete assignment: 
1. Load in Image Data into numpy arrays (`X`) 
2. Create a `y` for the labels
3. Train your model with pretrained layers from resnet
4. Report your model's accuracy

## Load in Data

![skimage-logo](https://scikit-image.org/_static/img/logo.png)

Check out out [`skimage`](https://scikit-image.org/) for useful functions related to processing the images. In particular checkout the documentation for `skimage.io.imread_collection` and `skimage.transform.resize`.

## Skimage experimentation

In [1]:
from skimage.io import imread_collection,imshow

In [2]:
a = imread_collection('data/mountain/art1121.jpg')

In [15]:
from skimage.data import load

a = load('astronaut.png')
print(a.shape)
a[0][:10]

(512, 512, 3)


array([[154, 147, 151],
       [109, 103, 124],
       [ 63,  58, 102],
       [ 54,  51,  98],
       [ 76,  76, 106],
       [100, 100, 104],
       [124, 121, 122],
       [139, 135, 133],
       [148, 141, 138],
       [141, 134, 130]], dtype=uint8)

In [23]:
from skimage.exposure import histogram
print(histogram(a)[0].shape)
histogram(a)[1].shape

(256,)


(256,)

In [2]:
import skimage.io as io
io.find_available_plugins()

{'fits': ['imread', 'imread_collection'],
 'gdal': ['imread', 'imread_collection'],
 'gtk': ['imshow'],
 'imageio': ['imread', 'imsave', 'imread_collection'],
 'imread': ['imread', 'imsave', 'imread_collection'],
 'matplotlib': ['imshow', 'imread', 'imshow_collection', 'imread_collection'],
 'pil': ['imread', 'imsave', 'imread_collection'],
 'qt': ['imshow', 'imsave', 'imread', 'imread_collection'],
 'simpleitk': ['imread', 'imsave', 'imread_collection'],
 'tifffile': ['imread', 'imsave', 'imread_collection']}

In [36]:
io.imread('data/mountain')

PermissionError: [Errno 13] Permission denied: 'data/mountain'

In [38]:
import os
data_files = os.listdir('./data/mountain')

In [47]:
filename = os.path.join('/data/mountain')

In [51]:
path = os.path.abspath('./data/mountain')

In [53]:
a = io.imread_collection(path)

In [59]:
help(io.ImageCollection)

Help on class ImageCollection in module skimage.io.collection:

class ImageCollection(builtins.object)
 |  ImageCollection(load_pattern, conserve_memory=True, load_func=None, **load_func_kwargs)
 |  
 |  Load and manage a collection of image files.
 |  
 |  Note that files are always stored in alphabetical order. Also note that
 |  slicing returns a new ImageCollection, *not* a view into the data.
 |  
 |  Parameters
 |  ----------
 |  load_pattern : str or list
 |      Pattern glob or filenames to load. The path can be absolute or
 |      relative.  Multiple patterns should be separated by os.pathsep,
 |      e.g. '/tmp/work/*.png:/tmp/other/*.jpg'.  Also see
 |      implementation notes below.
 |  conserve_memory : bool, optional
 |      If True, never keep more than one in memory at a specific
 |      time.  Otherwise, images will be cached once they are loaded.
 |  
 |  Other parameters
 |  ----------------
 |  load_func : callable
 |      ``imread`` by default.  See notes below.
 

## Successfully reading in the data:

In [1]:
import numpy as np
import skimage.io as io
mountain_dir = 'data/mountain/*.jpg'
forest_dir = 'data/forest/*.jpg'
mountain_imgs = io.imread_collection(mountain_dir)
forest_imgs = io.imread_collection(forest_dir)

In [2]:
print(len(forest_imgs))
print(len(mountain_imgs))

328
374


In [3]:
print(mountain_imgs[0].shape)
mountain_imgs[0][0]

(256, 256, 3)


Array([[  0,   4,  16],
       [  9,  22,  38],
       [ 38,  58,  83],
       [ 48,  73, 103],
       [ 39,  69, 105],
       [ 37,  71, 109],
       [ 35,  70, 112],
       [ 34,  71, 116],
       [ 34,  72, 119],
       [ 35,  73, 122],
       [ 36,  72, 124],
       [ 37,  73, 125],
       [ 36,  73, 125],
       [ 34,  74, 125],
       [ 34,  73, 128],
       [ 33,  74, 128],
       [ 33,  74, 128],
       [ 33,  74, 128],
       [ 33,  74, 128],
       [ 33,  74, 128],
       [ 32,  75, 128],
       [ 31,  74, 127],
       [ 33,  74, 128],
       [ 33,  74, 128],
       [ 32,  75, 128],
       [ 32,  75, 128],
       [ 33,  74, 128],
       [ 33,  74, 128],
       [ 32,  75, 128],
       [ 32,  75, 128],
       [ 30,  75, 130],
       [ 30,  75, 130],
       [ 31,  76, 131],
       [ 31,  76, 131],
       [ 33,  76, 131],
       [ 33,  76, 131],
       [ 31,  76, 131],
       [ 31,  76, 131],
       [ 33,  76, 131],
       [ 34,  77, 132],
       [ 33,  78, 133],
       [ 33,  78

In [4]:
img_shapes = []
for i in forest_imgs:
    img_shapes.append(i.shape)

In [5]:
# Making sure that all the images are the same shape:
print(len(img_shapes))
img_shapes[:10]

328


[(256, 256, 3),
 (256, 256, 3),
 (256, 256, 3),
 (256, 256, 3),
 (256, 256, 3),
 (256, 256, 3),
 (256, 256, 3),
 (256, 256, 3),
 (256, 256, 3),
 (256, 256, 3)]

In [6]:
# Checked here too, but here is the code in case
# img_shapes = []
# for i in mountain_imgs:
#     img_shapes.append(i.shape)
# # Making sure that all the images are the same shape:
# print(len(img_shapes))
# img_shapes[:10]

In [7]:
# now I need to make y_vectors in the shape of forests/mountaings 
# for the training:
x = []
for i in mountain_imgs:
    x.append(i)
for i in forest_imgs:
    x.append(i)

In [8]:
x = np.array(x)
x.shape

(702, 256, 256, 3)

In [9]:
type(x)

numpy.ndarray

In [10]:
y1 = np.zeros(len(forest_imgs),)
y2 = np.ones(len(mountain_imgs),)

y = np.append(y1,y2)
y.shape

(702,)

In [11]:
# Normalize data:
x_in = x/255

In [12]:
print(type(x_in))
x_in.shape

<class 'numpy.ndarray'>


(702, 256, 256, 3)

In [13]:
print('x:',x_in.shape,'y:',y.shape)

x: (702, 256, 256, 3) y: (702,)


## Instatiate Model

In [14]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input,decode_predictions
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

In [15]:
# Import resnet model without the fully connected top layers?
resnet = ResNet50(weights='imagenet',include_top=False,input_shape=(256,256,3))

Instructions for updating:
Colocations handled automatically by placer.


G:\Apps\Anaconda3\envs\U4-S3-DNN\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [16]:
for layer in resnet.layers:
    layer.trainable = False

In [17]:
x = resnet.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='selu')(x)
predictions = Dense(1,activation='sigmoid')(x)
model = Model(resnet.input,predictions)

In [18]:
model.compile(optimizer='nadam', loss='binary_crossentropy')

In [19]:
from tensorflow.keras import backend as K
dir(K.backend)

['__annotations__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '_tf_api_names',
 '_tf_api_names_v1']

## Fit Model

In [ ]:
model.fit(x_in,y,
          epochs=1,
          steps_per_epoch=5,
          batch_size=64
         )

Instructions for updating:
Use tf.cast instead.


# Custom CNN Model

In this step, write and train your own convolutional neural network using Keras. You can use any architecture that suits you as long as it has at least one convolutional and one pooling layer at the beginning of the network - you can add more if you want. 

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 215, 215, 32)      9632      
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 43, 43, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 39, 39, 64)        51264     
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 9, 9, 64)          102464    
_________________________________________________________________
flatten_1 (Flatten)          (None, 5184)              0         
_________________________________________________________________
dense_15 (Dense)             (None, 64)               

In [96]:
# Compile Model


In [99]:
# Fit Model


Train on 561 samples, validate on 141 samples
Epoch 1/5
561/561 [==============================] - 18s 32ms/sample - loss: 0.2667 - accuracy: 0.9073 - val_loss: 0.1186 - val_accuracy: 0.9858
Epoch 2/5
561/561 [==============================] - 18s 32ms/sample - loss: 0.2046 - accuracy: 0.9073 - val_loss: 0.3342 - val_accuracy: 0.8511
Epoch 3/5
561/561 [==============================] - 18s 32ms/sample - loss: 0.1778 - accuracy: 0.9287 - val_loss: 0.2746 - val_accuracy: 0.8723
Epoch 4/5
561/561 [==============================] - 18s 32ms/sample - loss: 0.1681 - accuracy: 0.9323 - val_loss: 0.8487 - val_accuracy: 0.5957
Epoch 5/5
561/561 [==============================] - 18s 32ms/sample - loss: 0.1606 - accuracy: 0.9394 - val_loss: 0.3903 - val_accuracy: 0.8582


# Custom CNN Model with Image Manipulations
## *This a stretch goal, and it's relatively difficult*

To simulate an increase in a sample of image, you can apply image manipulation techniques: cropping, rotation, stretching, etc. Luckily Keras has some handy functions for us to apply these techniques to our mountain and forest example. Check out these resources to help you get started: 

1. [Keras `ImageGenerator` Class](https://keras.io/preprocessing/image/#imagedatagenerator-class)
2. [Building a powerful image classifier with very little data](https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html)
 

In [ ]:
# State Code for Image Manipulation Here

# Resources and Stretch Goals

Stretch goals
- Enhance your code to use classes/functions and accept terms to search and classes to look for in recognizing the downloaded images (e.g. download images of parties, recognize all that contain balloons)
- Check out [other available pretrained networks](https://tfhub.dev), try some and compare
- Image recognition/classification is somewhat solved, but *relationships* between entities and describing an image is not - check out some of the extended resources (e.g. [Visual Genome](https://visualgenome.org/)) on the topic
- Transfer learning - using images you source yourself, [retrain a classifier](https://www.tensorflow.org/hub/tutorials/image_retraining) with a new category
- (Not CNN related) Use [piexif](https://pypi.org/project/piexif/) to check out the metadata of images passed in to your system - see if they're from a national park! (Note - many images lack GPS metadata, so this won't work in most cases, but still cool)

Resources
- [Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385) - influential paper (introduced ResNet)
- [YOLO: Real-Time Object Detection](https://pjreddie.com/darknet/yolo/) - an influential convolution based object detection system, focused on inference speed (for applications to e.g. self driving vehicles)
- [R-CNN, Fast R-CNN, Faster R-CNN, YOLO](https://towardsdatascience.com/r-cnn-fast-r-cnn-faster-r-cnn-yolo-object-detection-algorithms-36d53571365e) - comparison of object detection systems
- [Common Objects in Context](http://cocodataset.org/) - a large-scale object detection, segmentation, and captioning dataset
- [Visual Genome](https://visualgenome.org/) - a dataset, a knowledge base, an ongoing effort to connect structured image concepts to language